In [4]:
import pandas as pd
from mendeleev.fetch import fetch_table
from source.interactive_plots import interactive_histogram, interactive_violin
from source.get_elements      import get_elements

In [6]:
xrf_data = pd.read_csv("../data/interim/xrf_data_clean.csv")

In [7]:
# get elements analyzed
elements = get_elements(xrf_data.columns.tolist())

In [6]:
dropdown_buttons = {
    "categories": 
        [
            {
                "name": "Session", 
                "column": "date", 
                "button row": 1
            },
            {
                "name": "Group", 
                "column": "group", 
                "button row": 1
            }, 
            {
                "name": "Sample Type", 
                "column": "sample_type",
                "button row": 2
            },
            {
                "name": "QA/QC Type", 
                "column": "qaqc_type",
                "button row": 2
            }
        ], 
    "data": 
        {
            "name": "Elements", 
            "columns": elements
        }
    }

In [7]:
interactive_histogram(dropdown_buttons, xrf_data, x_axis_label='Concentration (ppm)', y_axis_label='No. of samples')

In [8]:
interactive_violin(dropdown_buttons, xrf_data, y_axis_label='Concentration (ppm)')
